## Análise da alavancagem do não-circulante

In [1]:
# bibliotecas usadas
import numpy as np
import pandas as pd

In [2]:
demonstracoes = pd.read_excel('D:\{estudos}\Python\ADC - Embraer S.A\Analise-Embraer\Demonstrações_Embraer_Ajustado.xlsm', sheet_name=None)
demonstracoes.keys()

<>:1: SyntaxWarning: invalid escape sequence '\{'
<>:1: SyntaxWarning: invalid escape sequence '\{'
C:\Users\Madu\AppData\Local\Temp\ipykernel_32384\71981632.py:1: SyntaxWarning: invalid escape sequence '\{'
  demonstracoes = pd.read_excel('D:\{estudos}\Python\ADC - Embraer S.A\Analise-Embraer\Demonstrações_Embraer_Ajustado.xlsm', sheet_name=None)


dict_keys(['Ativo', 'Passivo', 'DRE', 'DFC'])

In [ ]:
# Extração dos dados da planilha
if demonstracoes:
    df_ativo = demonstracoes.get('Ativo')
    df_passivo = demonstracoes.get('Passivo')
    df_dre = demonstracoes.get('DRE')

    # verificada manualmente a correspondência dos códigos com os demonstrativos
    contas_chave = {
        "AtivoCirculante": (df_ativo, "1.01"),
        "AtivoNaoCirculante": (df_ativo, "1.02"),
        "AtivoImobilizado": (df_ativo, "1.02.03"),
        "AtivoRealizavelLP": (df_ativo, "1.02.01"),
        "AtivoCirculante":      (df_ativo, "1.01"),
        "AtivoTotal": (df_ativo, "1"),
        "PassivoCirculante": (df_passivo, "2.01"),
        "PassivoNaoCirculante": (df_passivo, "2.02"),
        "PatrimonioLiquido": (df_passivo, "2.03"),
        "ReceitaLiquida": (df_dre, "3.01"),
        "LucroBruto": (df_dre, "3.03"),
        "LucroOperacional": (df_dre, "3.05"),
        "LucroLiquido": (df_dre, "3.11")
    }
    
    # Extrai os dados para um DataFrame organizado por ano
    anos = [col for col in df_ativo.columns if isinstance(col, (int, float))]
    base_data = pd.DataFrame(index=contas_chave.keys(), columns=anos)

    print("\n--- Extraindo valores-chave dos demonstrativos ---")
    for nome, (df, codigo) in contas_chave.items():
        if df is not None:
            # A busca agora ocorre no dataframe com colunas já limpas
            linha = df[df['CD_CONTA'] == str(codigo)] # Garante que o código seja string
            if not linha.empty:
                valores = linha.iloc[0][anos]
                base_data.loc[nome] = valores
            else:
                print(f"   AVISO: Conta '{nome}' (código {codigo}) não encontrada. Será preenchida com 0.")
                base_data.loc[nome] = 0
    
    # Calcula totais necessários
    base_data.loc['PassivoTotal'] = base_data.loc['PassivoCirculante'] + base_data.loc['PassivoNaoCirculante']
    base_data.loc['CapitalPermanente'] = base_data.loc['PatrimonioLiquido'] + base_data.loc['PassivoNaoCirculante']
    base_data.loc['AtivoPermanente'] = base_data.loc['AtivoImobilizado']

    print("\n--- Base de Dados para Cálculo ---")
    print(base_data.to_string())


--- Extraindo valores-chave dos demonstrativos ---

--- Base de Dados para Cálculo ---
                                   2020             2021             2022             2023        2024
AtivoCirculante         40549746.680452  37967991.536853  32876200.424879     30670519.513    40326906
AtivoNaoCirculante      29141300.635077  27650888.976054  25044728.794734    23942405.2206    32892812
AtivoImobilizado        13366071.253672  11293848.893319   9790271.260124     9414354.1516    12669526
AtivoRealizavelLP        1995054.146193   2026121.285432   2354678.507979      2578961.465     4454043
AtivoTotal              69691047.315529  65618880.512908  57920929.219613    54612924.7336    73219718
PassivoCirculante        16084627.91635  18279637.446943  18373669.443463    18565945.7574  27347016.0
PassivoNaoCirculante    34305057.823225  29409556.675324  23417345.436912    20652951.5502  25161794.0
PatrimonioLiquido       19301361.575954   17929686.39064  16129914.339238     15394027.4

### Calculando os indicadores do não circulante

In [12]:
indicadores = pd.DataFrame(columns=anos) # DataFrame para os resultados

# =============================================================================
# INDICADORES DE ESTRUTURA DE CAPITAL (NÃO CIRCULANTE)

# 1. Endividamento Total
indicadores.loc['Endividamento Total'] = base_data.loc['PassivoTotal'] / base_data.loc['PatrimonioLiquido']

# 2. Dependência/Independência Financeira
indicadores.loc['Dependência Financeira (%)'] = (base_data.loc['PassivoTotal'] / base_data.loc['AtivoTotal']) * 100
indicadores.loc['Independência Financeira (%)'] = (base_data.loc['PatrimonioLiquido'] / base_data.loc['AtivoTotal']) * 100

# 3. Participação de Capital de Terceiros LP
indicadores.loc['Capital Terceiros LP'] = base_data.loc['PassivoNaoCirculante'] / base_data.loc['PatrimonioLiquido']

# =============================================================================
# INDICADORES DE IMOBILIZAÇÃO

# 4. Imobilização do Capital Permanente
indicadores.loc['Imobilização Capital Permanente'] = base_data.loc['AtivoNaoCirculante'] / base_data.loc['CapitalPermanente']

# 5. Imobilização do Patrimônio Líquido
indicadores.loc['Imobilização PL'] = base_data.loc['AtivoPermanente'] / base_data.loc['PatrimonioLiquido']

# 6. Imobilização de Recursos Não Correntes
indicadores.loc['Imobilização Recursos Não Correntes'] = (
    (base_data.loc['AtivoNaoCirculante'] - base_data.loc['AtivoRealizavelLP']) / 
    base_data.loc['CapitalPermanente']
)


# =============================================================================
# INDICADORES DE DESEMPENHO DO IMOBILIZADO

# 7. Giro do Imobilizado
indicadores.loc['Giro do Imobilizado'] = base_data.loc['ReceitaLiquida'] / base_data.loc['AtivoImobilizado']

# 8. Nível de Automação (simplificado - usando apenas imobilizado)
# Nota: Para cálculo completo precisa do número de operários
indicadores.loc['Nível Automação (Imobilizado/Receita)'] = (
    base_data.loc['AtivoImobilizado'] / base_data.loc['ReceitaLiquida']
)

# =============================================================================
# INDICADORES DE LIQUIDEZ E COMPOSIÇÃO

# 9. Liquidez Geral
indicadores.loc['Liquidez Geral'] = (
    (base_data.loc['AtivoCirculante'] + base_data.loc['AtivoRealizavelLP']) / 
    (base_data.loc['PassivoCirculante'] + base_data.loc['PassivoNaoCirculante'])
)

# 10. Composição do Endividamento (Longo Prazo)
indicadores.loc['Composição Endividamento LP (%)'] = (
    base_data.loc['PassivoNaoCirculante'] / base_data.loc['PassivoTotal']
) * 100

# =============================================================================
# INDICADORES ADICIONAIS RELEVANTES

# Participação do Não Circulante no Ativo Total
indicadores.loc['Participação ANC no Ativo (%)'] = (
    base_data.loc['AtivoNaoCirculante'] / base_data.loc['AtivoTotal']
) * 100

# Alavancagem Financeira (PNC/PL)
indicadores.loc['Alavancagem Financeira (PNC/PL)'] = (
    base_data.loc['PassivoNaoCirculante'] / base_data.loc['PatrimonioLiquido']
)

print("\n\n--- INDICADORES DE ANÁLISE DO NÃO CIRCULANTE ---")
print("=" * 60)


print("\n\n--- Tabela Final de Indicadores Financeiros ---")
print(indicadores.round(2).to_string())



--- INDICADORES DE ANÁLISE DO NÃO CIRCULANTE ---


--- Tabela Final de Indicadores Financeiros ---
                                            2020       2021       2022       2023       2024
Endividamento Total                      2.61068    2.65979   2.590901    2.54767   2.535321
Dependência Financeira (%)             72.304389  72.676025  72.151838  71.812483  71.714029
Independência Financeira (%)           27.695611  27.323975  27.848162  28.187517  28.285971
Capital Terceiros LP                    1.777339   1.640272   1.451796   1.341621   1.214905
Imobilização Capital Permanente         0.543616   0.584101   0.633286     0.6642   0.717045
Imobilização PL                         0.692494   0.629897   0.606964   0.611559   0.611732
Imobilização Recursos Não Correntes     0.506399   0.541301   0.573745   0.592656    0.61995
Giro do Imobilizado                      1.70785   2.214638   2.517534   2.912558   2.796014
Nível Automação (Imobilizado/Receita)   0.585531   0.451541   

In [13]:
# Formata a saída para melhor visualização
for indicador in indicadores.index:
    print(f"\n{indicador}:")
    for ano in indicadores.columns:
        valor = indicadores.loc[indicador, ano]
        if any(termo in indicador for termo in ['%', 'Percent', 'Composição', 'Dependência', 'Independência', 'Participação']):
            print(f"  {ano}: {valor:.2f}%")
        elif 'Vida Útil' in indicador:
            print(f"  {ano}: {valor:.1f} anos")
        else:
            print(f"  {ano}: {valor:.4f}")

# Tabela resumo
print("\n\n--- TABELA RESUMO DOS INDICADORES ---")
print(indicadores.round(4).to_string())


Endividamento Total:
  2020: 2.6107
  2021: 2.6598
  2022: 2.5909
  2023: 2.5477
  2024: 2.5353

Dependência Financeira (%):
  2020: 72.30%
  2021: 72.68%
  2022: 72.15%
  2023: 71.81%
  2024: 71.71%

Independência Financeira (%):
  2020: 27.70%
  2021: 27.32%
  2022: 27.85%
  2023: 28.19%
  2024: 28.29%

Capital Terceiros LP:
  2020: 1.7773
  2021: 1.6403
  2022: 1.4518
  2023: 1.3416
  2024: 1.2149

Imobilização Capital Permanente:
  2020: 0.5436
  2021: 0.5841
  2022: 0.6333
  2023: 0.6642
  2024: 0.7170

Imobilização PL:
  2020: 0.6925
  2021: 0.6299
  2022: 0.6070
  2023: 0.6116
  2024: 0.6117

Imobilização Recursos Não Correntes:
  2020: 0.5064
  2021: 0.5413
  2022: 0.5737
  2023: 0.5927
  2024: 0.6199

Giro do Imobilizado:
  2020: 1.7079
  2021: 2.2146
  2022: 2.5175
  2023: 2.9126
  2024: 2.7960

Nível Automação (Imobilizado/Receita):
  2020: 0.5855
  2021: 0.4515
  2022: 0.3972
  2023: 0.3433
  2024: 0.3577

Liquidez Geral:
  2020: 0.8443
  2021: 0.8386
  2022: 0.8430
  2023

### Exportar a tabela para excel

In [14]:
nome_arquivo_saida = "Indicadores_Financeiros_Embraer.xlsx"
try:
    # Cria um "escritor" de Excel para salvar múltiplas abas
    with pd.ExcelWriter(nome_arquivo_saida, engine='openpyxl') as writer:
        indicadores.to_excel(writer, sheet_name='Indicadores')
        base_data.to_excel(writer, sheet_name='Dados Base')
            
    print(f"\n\n Tabela de indicadores exportada com sucesso para o arquivo: '{nome_arquivo_saida}'")
    print("   O arquivo contém duas abas: 'Indicadores' e 'Dados Base'.")
except Exception as e:
    print(f"\n ERRO ao exportar para Excel: {e}")



 Tabela de indicadores exportada com sucesso para o arquivo: 'Indicadores_Financeiros_Embraer.xlsx'
   O arquivo contém duas abas: 'Indicadores' e 'Dados Base'.


## Análise dos indicadores do não circulante

### `Indicadores Consistentes`

### - Estrutura de Capital
- **Endividamento Total (~2.5-2.6)**: Muito alto - para cada R$1 de PL, há R$2,5-2,6 de dívida
- **Dependência Financeira (~72%)**: 72% do ativo é financiado por terceiros (preocupante)
- **Independência Financeira (~28%)**: Apenas 28% com recursos próprios (baixo)

### - Composição do Endividamento
- **Composição LP (%)**: Caiu de 68% para 48% - migrando dívidas para curto prazo (risco)

### - Participação ANC
- **~42-45% do Ativo Total**: Valor razoável para empresa industrial

### `Indicadores com Possíveis Problemas`

### - Liquidez Geral (~0.84-0.85)
**PROBLEMA GRAVE**: Valor abaixo de 1.0 indica que:
- Ativos de curto e longo prazo NÃO cobrem obrigações totais
- Risco de insolvência se credores exigirem pagamento

### - Giro do Imobilizado (1.7-2.9)
Valores baixos para empresa industrial. Ideal seria > 3.0

### - Capital Terceiros LP (1.2-1.8)
Muito elevado - dívida de LP maior que o PL